In [404]:
# Import needed libraries 

import pandas as pd 
import os
import datetime
from datetime import timedelta
import numpy as np 
from scipy.signal import argrelextrema
import alpaca_trade_api as tradeapi 
import matplotlib.pyplot as plt 
import matplotlib.dates as mpdates
from mplfinance.original_flavor import candlestick_ohlc
from dotenv import load_dotenv
from itertools import islice
import hvplot.pandas

In [405]:
# Load .env file

load_dotenv()

True

In [406]:
# Set Alpaca API key and secret passwords

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [407]:
# Initiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [408]:
# Parameters for Stock Data from Alpacas
# Establish time frame (5 minute)

time_frame = "5min"

# Identify what stock symbol is trading

stock_symbol = "QQQ"

# Identify what start date to begin data analysis

start_date = pd.Timestamp("2021-02-01", tz="America/New_York").isoformat()

# Identify what end date to finalize data analysis

end_date = pd.Timestamp("2021-03-01", tz="America/New_York").isoformat()

In [409]:
# Function to call daily stock data

def get_stock_data(api, stock_symbol, time_frame, current_date_iso, next_day_date_iso):
    
    # Assuming api.get_bars returns a DataFrame with a 'df' attribute
    
    stock_data = api.get_bars(
        stock_symbol, 
        time_frame, 
        start=current_date_iso, 
        end=next_day_date_iso
        ).df
    
    return stock_data

# Displays the information pulled for working through code

# stock_data = get_stock_data(api, stock_symbol, time_frame, start_date, end_date)
# stock_data.info()
# display(stock_data.head())
# display(stock_data.tail())


In [410]:
# Function to prepare daily stock data to identify double top/bottom patterns and prepare for targets
# Includes establishing a polynomial fit and assigning new columns for localized min/max

# Polynomial Degree

polynomial_degree = 25

def polynomial_min_max_fit(stock_data, polynomial_degree):
    
    min_length = len(stock_data.index)
    x_data = np.arange(min_length)

    # Polynomial fitting
    polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
    polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
    polynomial_coefficients_low = np.polyfit(x_data, stock_data['low'][:min_length], polynomial_degree)
    polynomial_coefficients_close = np.polyfit(x_data, stock_data['close'][:min_length], polynomial_degree)

    # Evaluate the polynomial fit for plotting
    y_polynomial_open = np.polyval(polynomial_coefficients_open, x_data)
    y_polynomial_high = np.polyval(polynomial_coefficients_high, x_data)
    y_polynomial_low = np.polyval(polynomial_coefficients_low, x_data)
    y_polynomial_close = np.polyval(polynomial_coefficients_close, x_data)

    # Identify local extrema for polynomial fit data (minima and maxima)
    local_poly_minima = argrelextrema(y_polynomial_close, np.less, order=5)[0]
    local_poly_maxima = argrelextrema(y_polynomial_close, np.greater, order=5)[0]

    # Convert the close price polynomial fit data into a dataframe
    # This is done for OHLC poly fit data

    poly_df = pd.DataFrame(y_polynomial_open)
    columns = ['poly_fit_open']
    poly_df.columns = columns
    poly_df = poly_df.assign(poly_fit_high = y_polynomial_high)
    poly_df = poly_df.assign(poly_fit_low = y_polynomial_low)
    poly_df = poly_df.assign(poly_fit_close = y_polynomial_close)

    # Reset the index of the original updated ticker dataframe to concat with the polynomial dataframe that does not include a timeseries
    # This will ensure that the indexed intergers of the ploynomial fit align with the time each data point corresponds to

    updated_stock_data = stock_data.reset_index()
    updated_stock_data = pd.concat([updated_stock_data, poly_df], axis='columns', join='inner')
    updated_stock_data.head()

    # Add minima and maxima column to the DataFrame

    updated_stock_data["minima"] = 0
    updated_stock_data["maxima"] = 0
    updated_stock_data.head()

    # Mark rows with local minima as 1 in the 'minima' column

    for index in local_poly_minima:
        updated_stock_data.at[index, 'minima'] = -1

    for index in local_poly_maxima:
        updated_stock_data.at[index, "maxima"] = 1

    # Create Target Columns - Double Top Target & Double Bottom Target
        
    updated_stock_data["dbl_top_target"] = 0
    updated_stock_data["dbl_bot_target"] = 0

    # # Plot data: COMMENTED OUT TO NOT REPEAT PLOTS AT END OF PROGRAM
    # # Plot the stock data and identified minima

    # plt.figure(figsize=(15, 5))
    # plt.plot(stock_data.index, stock_data["close"], label='Close Prices', alpha=0.7)

    # # Plot polynomial fit

    # plt.plot(stock_data.index[:min_length], y_polynomial_close, '-', markersize=1.0, color='black', alpha=0.9, label='Polynomial Fit')

    # # Plot red dots at local minima and blue dots at local maxima

    # plt.scatter(stock_data.index[local_poly_minima], y_polynomial_close[local_poly_minima], color='red', label='Local Minima')
    # plt.scatter(stock_data.index[local_poly_maxima],y_polynomial_close[local_poly_maxima], color="blue", label = "Local Maxima")
    
    return updated_stock_data

# For data checking/confirmation

# updated_stock_data = polynomial_min_max_fit(stock_data,polynomial_degree)

In [411]:
# # Plot data
# # Plot the stock data and identified minima

# plt.figure(figsize=(15, 5))
# plt.plot(stock_data.index, stock_data["close"], label='Close Prices', alpha=0.7)

# # # Plot polynomial fit

# plt.plot(stock_data.index[:min_length], y_polynomial_close, '-', markersize=1.0, color='black', alpha=0.9, label='Polynomial Fit')

# # # Plot red dots at local minima and blue dots at local maxima

# plt.scatter(stock_data.index[local_poly_minima], y_polynomial_close[local_poly_minima], color='red', label='Local Minima')
# plt.scatter(stock_data.index[local_poly_maxima],y_polynomial_close[local_poly_maxima], color="blue", label = "Local Maxima")

In [412]:
# Define the time independent DF for double top/bottom identification
# Timestamp remains a column, but can identify pattern from peak to peak immediately

def time_independent_data(updated_stock_data):

    # Define and establish time independent DF

    time_independent_df = []
    time_independent_df = pd.DataFrame(time_independent_df, columns = ["timestamp","close","high","low","trade_count","open","volume","vwap","poly_fit_open","poly_fit_high","poly_fit_low","poly_fit_close","minima","maxima","dbl_top_target","dbl_bot_target"])

    # Loop to iterate through all rows of stock data and write min/max to new DF

    for index, row in islice(updated_stock_data.iterrows(), 0, None):

        # Assign to rows only those that contain local min or max

        if (updated_stock_data.at[index,"minima"] == -1):
            time_independent_df.loc[index] = row
            a = updated_stock_data.iloc[index]["poly_fit_close"]
        elif (updated_stock_data.at[index,"maxima"] == 1):
            time_independent_df.loc[index] = row
            a = updated_stock_data.iloc[index]["poly_fit_close"]

    # Reset time dependent index
    # Set new time independent index

    time_independent_df.reset_index(inplace = True)
    time_independent_df.rename(columns={"index":"time_dependent_index"}, inplace = True)

    return time_independent_df

# Data confirmation/check

# time_independent_df = time_independent_data(updated_stock_data)
# time_independent_df.head(10)

In [413]:
# Function to identify daily double top/bottom patterns
# Inherently there should only be 1 identification of the pattern as it is considered a trend reversal pattern
# The loop will break once the pattern is identified

def identify_double_patterns(time_independent_df, updated_stock_data):

    # Initialize variables to identify double top/bottom patterns
    # Time dependent variable x_0 will always begin at a local min/max which also coincides
    # With the start of a trend into a potential double top/bottom
    
    x_0 = 0

    # Initiation of double top/bottom variable
    
    a = 0

    # First peak/valley of double top/bottom pattern
    
    b = 0

    # Trough/peak of double top/bottom pattern

    c = 0

    # Second peak/vallye of double top/bottom pattern
    
    d = 0

    # Trigger of double top/bottom pattern
    
    e = 0

    # Final time increment to finalize and trigger double top signal
    
    x_f = 0

    # Read through code to identify double top/bottom and assign to target columns.

    for index, row in islice(time_independent_df.iterrows(), 0, len(time_independent_df) - 4):

        # Check for double top
        # If found then assigns x_f final time for writing to targets in time dependent dataframe

        if (time_independent_df.at[index,"minima"] == -1):
            a = time_independent_df.iloc[index]["poly_fit_close"]
            b = time_independent_df.iloc[index + 1]["poly_fit_close"]
            c = time_independent_df.iloc[index + 2]["poly_fit_close"]
            d = time_independent_df.iloc[index + 3]["poly_fit_close"]
            e = time_independent_df.iloc[index + 4]["poly_fit_close"]
            x_0 = time_independent_df.iloc[index]["time_dependent_index"]

            if (time_independent_df.iloc[index + 1]["poly_fit_low"])*.95 < d < (time_independent_df.iloc[index + 1]["poly_fit_high"])*1.05 and (e <= c):
                x_f = time_independent_df.iloc[index + 4]["time_dependent_index"]
                while (x_0 < x_f + 1):
                    updated_stock_data.at[x_0, "dbl_top_target"] = 1
                    x_0 = x_0 + 1
                break
            
            

        # Check for double bottom
        # If found then assigns x_f final time for writing to targets in time dependent dataframe
            
        elif (time_independent_df.at[index,"maxima"] == 1):
            a = time_independent_df.iloc[index]["poly_fit_close"]
            b = time_independent_df.iloc[index + 1]["poly_fit_close"]
            c = time_independent_df.iloc[index + 2]["poly_fit_close"]
            d = time_independent_df.iloc[index + 3]["poly_fit_close"]
            e = time_independent_df.iloc[index + 4]["poly_fit_close"]
            x_0 = time_independent_df.iloc[index]["time_dependent_index"] 

            if (time_independent_df.iloc[index+1]["poly_fit_low"])*.95 < d < (time_independent_df.iloc[index+1]["poly_fit_high"])*1.05 and (e >= c):
                x_f_min = time_independent_df.iloc[index + 4]["time_dependent_index"]
                while (x_0 < x_f + 1):
                    updated_stock_data.at[x_0, "dbl_bot_target"] = 1
                    x_0 = x_0 + 1
                break
            
    return updated_stock_data, x_f

# updated_stock_data, x_f = identify_double_patterns(time_independent_df,updated_stock_data)

# print(x_f)



In [414]:
# # Data Checking

# display(updated_stock_data.head(20))
# print(updated_stock_data.loc[x_f])

In [416]:
# Loop to collect and assess daily data one day at a time

master_df = {
    'timestamp': [],  # List of timestamps
    'close': [],      # List of close prices
    'high': [],       # List of high prices
    'low': [],        # List of low prices
    'trade_count': [], # List of trade counts
    'open': [],       # List of open prices
    'volume': [],     # List of volumes
    'vwap': [],       # List of volume-weighted average prices
    'poly_fit_open': [],    # List of polynomial fit open prices
    'poly_fit_high': [],    # List of polynomial fit high prices
    'poly_fit_low': [],     # List of polynomial fit low prices
    'poly_fit_close': [],   # List of polynomial fit close prices
    'minima': [],           # List of minima
    'maxima': [],           # List of maxima
    'dbl_top_target': [],   # List of double top targets
    'dbl_bot_target': []    # List of double bottom targets
}

daily_dataframes = []

master_df = pd.DataFrame(master_df)

current_date = pd.to_datetime(start_date)
next_day_date = current_date + pd.offsets.BDay(1)

print(current_date)
print(next_day_date)

while current_date <= (pd.to_datetime(end_date)):

    # next_day_date = next_day_date + pd.offsets.BDay(1)
    # current_date = current_date + pd.offsets.BDay(1)
    # Convert current_date and next_day_date to ISO format for API call

    current_date_iso = current_date.isoformat()
    # next_day_date = current_date + pd.offsets.BDay(1)
    next_day_date_iso = next_day_date.isoformat()

    #current_date = datetime.datetime.fromisoformat(current_date_iso) + pd.offsets.BDay(1)
    
    # Fetch stock data for the current day

    stock_data = get_stock_data(api, stock_symbol, time_frame, current_date_iso, next_day_date_iso)
    
    # Run polynomial fit function for updated stock data

    updated_stock_data = polynomial_min_max_fit(stock_data, polynomial_degree)
    
    # Isolate Mins/maxes

    time_independent_df = time_independent_data(updated_stock_data)

    # Updated stock data with double tops/bots identified by function
    # x_f is the double top/bot trigger row(index) for current day's double top/bot

    updated_stock_data, x_f = identify_double_patterns(time_independent_df, updated_stock_data)
    
    # Perform your analysis or call your functions here
    # e.g., identify_double_patterns(time_independent_df, updated_stock_data)

    daily_dataframes.append(updated_stock_data)

    master_df = pd.concat(daily_dataframes, ignore_index=True)
    display(master_df.tail())
    # Increment to the next day

    next_day_date = next_day_date + pd.offsets.BDay(1)
    current_date = current_date + pd.offsets.BDay(1)
    # next_day_date = pd.Timestamp(next_day_date_iso) + pd.offsets.BDay(1)
    
    print(current_date)
    print(next_day_date)
    


2021-02-01 00:00:00-05:00
2021-02-02 00:00:00-05:00


/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_8136/1656766669.py:14: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_8136/1656766669.py:15: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_8136/1656766669.py:16: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_low = np.polyfit(x_data, stock_data['low'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_8136/1656766669.py:17: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_close = np.polyfit(x_data, stock_data['close'][:min_length], polynomial_degree)


,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,minima,maxima,dbl_top_target,dbl_bot_target
187,2021-02-02 00:35:00+00:00,322.94,323.00,322.93,41,322.97,9725,322.942734,322.775932,322.922946,322.712112,322.874416,0,0,0,0
188,2021-02-02 00:40:00+00:00,322.99,323.00,322.71,32,322.82,1554,322.861236,322.893843,322.985449,322.811880,322.932437,0,0,0,0
189,2021-02-02 00:45:00+00:00,322.89,322.99,322.82,54,322.99,8625,322.890602,323.027878,323.048827,322.922654,322.986772,0,0,0,0
190,2021-02-02 00:50:00+00:00,322.85,322.97,322.84,39,322.93,4724,322.885571,323.068478,323.069413,322.958433,323.005532,0,1,0,0
191,2021-02-02 00:55:00+00:00,323.05,323.05,322.85,72,322.89,12541,322.965053,322.805153,322.986185,322.756471,322.950314,0,0,0,0


2021-02-02 00:00:00-05:00
2021-02-03 00:00:00-05:00


/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_8136/1656766669.py:14: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_8136/1656766669.py:15: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_8136/1656766669.py:16: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_low = np.polyfit(x_data, stock_data['low'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_8136/1656766669.py:17: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_close = np.polyfit(x_data, stock_data['close'][:min_length], polynomial_degree)


...

2021-02-12 00:00:00-05:00
2021-02-15 00:00:00-05:00


/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_8136/1656766669.py:14: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_open = np.polyfit(x_data, stock_data['open'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_8136/1656766669.py:15: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_high = np.polyfit(x_data, stock_data['high'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_8136/1656766669.py:16: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_low = np.polyfit(x_data, stock_data['low'][:min_length], polynomial_degree)
/var/folders/sr/78cv2dx14kgdcwtf90634z580000gn/T/ipykernel_8136/1656766669.py:17: RankWarning: Polyfit may be poorly conditioned
  polynomial_coefficients_close = np.polyfit(x_data, stock_data['close'][:min_length], polynomial_degree)


,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,minima,maxima,dbl_top_target,dbl_bot_target
1854,2021-02-12 22:35:00+00:00,336.82,336.82,336.82,7,336.82,3293,336.483167,336.783712,336.780239,336.806586,336.788081,0,0,0,0
1855,2021-02-12 22:40:00+00:00,336.78,336.80,336.78,5,336.80,1009,336.789316,336.792808,336.795342,336.804902,336.799764,0,0,0,0
1856,2021-02-12 22:45:00+00:00,336.80,336.80,336.78,5,336.78,1060,336.789302,336.788659,336.808188,336.774850,336.801068,0,1,0,0
1857,2021-02-12 22:50:00+00:00,336.80,336.80,336.80,16,336.80,680,336.793339,336.780074,336.810749,336.747058,336.794542,0,0,0,0
1858,2021-02-12 22:55:00+00:00,336.78,336.78,336.78,41,336.78,596,336.743859,336.791090,336.776180,336.806803,336.783281,0,0,0,0


2021-02-15 00:00:00-05:00
2021-02-16 00:00:00-05:00


KeyError: 'open'

In [ ]:
master_df.info()
print(len(master_df))
print(type(master_df))
print(sum(master_df["dbl_top_target"]))
print(sum(master_df["dbl_bot_target"]))
display(master_df.head(50))
display(master_df.tail(50))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4113 entries, 0 to 4112
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   timestamp       4113 non-null   datetime64[ns, UTC]
 1   close           4113 non-null   float64            
 2   high            4113 non-null   float64            
 3   low             4113 non-null   float64            
 4   trade_count     4113 non-null   int64              
 5   open            4113 non-null   float64            
 6   volume          4113 non-null   int64              
 7   vwap            4113 non-null   float64            
 8   poly_fit_open   4113 non-null   float64            
 9   poly_fit_high   4113 non-null   float64            
 10  poly_fit_low    4113 non-null   float64            
 11  poly_fit_close  4113 non-null   float64            
 12  minima          4113 non-null   int64              
 13  maxima          4113 non-null   i

,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,minima,maxima,dbl_top_target,dbl_bot_target
0,2021-02-02 09:00:00+00:00,325.16,325.20,324.86,40,324.97,4482,325.010297,324.937940,325.269611,324.839743,325.221107,0,0,0,0
1,2021-02-02 09:05:00+00:00,325.29,325.33,324.88,30,324.92,5726,325.114829,325.082603,325.316479,325.015084,325.263234,0,0,0,0
2,2021-02-02 09:10:00+00:00,325.50,325.56,325.31,25,325.31,4047,325.456284,325.173133,325.356887,325.124463,325.300034,0,0,0,0
3,2021-02-02 09:15:00+00:00,325.25,325.44,325.25,30,325.39,4701,325.350742,325.248849,325.408655,325.208555,325.347449,0,0,0,0
4,2021-02-02 09:20:00+00:00,325.40,325.40,325.19,13,325.26,2485,325.290137,325.324197,325.472982,325.285496,325.406846,0,0,0,0
5,2021-02-02 09:25:00+00:00,325.44,325.44,325.41,7,325.41,1005,325.424925,325.400693,325.543745,325.360667,325.473020,0,0,0,0
6,2021-02-02 09:30:00+00:00,325.27,325.31,325.27,3,325.31,500,325.278000,325.474432,325.613061,325.433125,325.539029,0,0,0,0
7,2021-02-02 09:35:00+00:00,325.64,325.71,325.42,25,325.45,4253,325.512995,325.540476,325.674253,325.499605,325.598808,0,0,0,0
8,2021-02-02 09:40:00+00:00,325.74,325.89,325.55,33,325.55,4278,325.777415,325.595071,325.723103,325.556795,325.648308,0,0,0,0
9,2021-02-02 09:45:00+00:00,325.73,325.73,325.73,2,325.73,115,325.735217,325.636481,325.758020,325.602444,325.685715,0,0,0,0


,timestamp,close,high,low,trade_count,open,volume,vwap,poly_fit_open,poly_fit_high,poly_fit_low,poly_fit_close,minima,maxima,dbl_top_target,dbl_bot_target
4063,2021-04-01 19:45:00+00:00,324.31,324.400,324.0300,5716,324.04,1205879,323.267016,324.130555,324.327528,324.012901,324.228184,0,0,0,0
4064,2021-04-01 19:50:00+00:00,324.52,324.520,324.1610,7979,324.31,1162935,324.319953,324.212993,324.415197,324.099784,324.328327,0,0,0,0
4065,2021-04-01 19:55:00+00:00,324.55,324.680,324.3300,13413,324.51,2626618,323.628391,324.307551,324.509770,324.199723,324.435110,0,0,0,0
4066,2021-04-01 20:00:00+00:00,324.62,324.690,324.5300,582,324.55,1087471,324.585373,324.409268,324.606065,324.307889,324.542560,0,0,0,0
4067,2021-04-01 20:05:00+00:00,324.83,324.850,324.6600,405,324.68,1844764,324.632614,324.512527,324.698645,324.418651,324.644563,0,0,0,0
4068,2021-04-01 20:10:00+00:00,324.83,324.830,324.7100,219,324.82,56229,324.774540,324.611520,324.782284,324.526028,324.735413,0,0,0,0
4069,2021-04-01 20:15:00+00:00,324.74,324.890,324.5700,127,324.57,29816,324.774548,324.700766,324.852446,324.624194,324.810314,0,0,0,0
4070,2021-04-01 20:20:00+00:00,324.65,324.800,324.5500,105,324.70,23678,324.685307,324.775601,324.905701,324.707989,324.865844,0,0,0,0
4071,2021-04-01 20:25:00+00:00,324.72,324.721,324.6601,27,324.70,1224,324.693844,324.832608,324.940064,324.773399,324.900273,0,0,0,0
4072,2021-04-01 20:30:00+00:00,324.89,324.890,324.5700,99,324.68,124800,324.615780,324.869937,324.955182,324.817963,324.913734,0,1,0,0


In [ ]:
print(current_date)

2021-04-02 00:00:00-05:00


In [ ]:
master_df.hvplot(x = "timestamp")



/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/Users/ebrown/anaconda3/envs/machine_learning/lib/python3.10/site-packages/holoviews/core/data/pandas.py:362: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series conta

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)